<a href="https://colab.research.google.com/github/rkat7/practice-cuda/blob/main/starting_cuda_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pwd
%cd drive
#applying the cd to the entire notebook using %
%cd MyDrive
%mkdir practice-cuda
%cd practice-cuda
%pwd



/content/drive/MyDrive
[Errno 2] No such file or directory: 'drive'
/content/drive/MyDrive
[Errno 2] No such file or directory: 'MyDrive'
/content/drive/MyDrive
/content/drive/MyDrive/practice-cuda


'/content/drive/MyDrive/practice-cuda'

In [ ]:
!which nvcc #checking if cuda compiler is installed/present in the environment
!ls -l /dev/nv* #Checking if an Nvidia gpu is mounted to current colab environment

/usr/local/cuda/bin/nvcc
crw-rw-rw- 1 root root 195,   0 Feb 12 03:13 /dev/nvidia0
crw-rw-rw- 1 root root 195, 255 Feb 12 03:13 /dev/nvidiactl
crw-rw-rw- 1 root root 241,   0 Feb 12 03:13 /dev/nvidia-uvm
crw-rw-rw- 1 root root 241,   1 Feb 12 03:13 /dev/nvidia-uvm-tools

/dev/nvidia-caps:
total 0
cr-------- 1 root root 244, 1 Feb 12 03:13 nvidia-cap1
cr--r--r-- 1 root root 244, 2 Feb 12 03:13 nvidia-cap2


In [ ]:
!nvcc test.cu -o test

In [ ]:
!./test

Hello from CPU! 


In [ ]:
!nvidia-smi

Mon Sep 15 04:27:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# CUDA device global memory management APIs:

#cudaMalloc()
#cudaFree()

%%writefile device_mem_sample.cu
#include <stdio.h>

int main(){

float *A_d;
int size = 4 * sizeof(float);

cudaMalloc((void**) &A_d, size);

//some computations
cudaFree(A_d);

return 0;
}


Overwriting device_mem_sample.cu


In [ ]:
!nvcc device_mem_sample.cu -o device_mem_sample

In [ ]:
!./device_mem_sample

In [ ]:
#cudaMemcpy()

%%writefile device_mem_sample.cu

#include<stdio.h>

int main(){

float *A_d, *A_h;

// assuming the _h suffixed variables are already initialized on the host


int size = 4 * sizeof(float);

cudaMalloc((void**) &A_d, size);
cudaMemcpy(A_d, A_h, size, cudaMemcpyHostToDevice); //another option is cudaMemcpyDeviceToHost

//some computations

cudaFree(A_d);

return 0;
}


Writing device_mem_sample.cu


In [ ]:
!nvcc device_mem_sample.cu -o device_mem_sample

device_mem_sample.cu(14): warning #549-D: variable "A_h" is used before its value is set
  cudaMemcpy(A_d, A_h, size, cudaMemcpyHostToDevice);
                  ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

device_mem_sample.cu(14): warning #549-D: variable "A_h" is used before its value is set
  cudaMemcpy(A_d, A_h, size, cudaMemcpyHostToDevice);
                  ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [ ]:
!./device_mem_sample

**Simple Vector Addition:**


In [ ]:
%%writefile vecAdd.cu

#include <stdio.h>

__global__
void vecAddKernel(float* A, float* B, float* C, int n) {
  int i = threadIdx.x + blockDim.x * blockIdx.x;
  if (i < n){
    C[i] = A[i] + B[i];
  }
}

//DOn't need __host__ because by default all functions are host functions in cuda

void vecAdd(float* A, float* B, float* C, int n){
  float *A_d, *B_d, *C_d;
  cudaMalloc((void**) &A_d, n * sizeof(float));
  cudaMalloc((void**) &B_d, n * sizeof(float));
  cudaMalloc((void**) &C_d, n * sizeof(float));

  cudaMemcpy(A_d, A, n, cudaMemcpyHostToDevice);
  cudaMemcpy(B_d, B, n, cudaMemcpyHostToDevice);

  vecAddKernel<<<ceil(n/256.0), 256>>>(A_d, B_d, C_d, n);

  cudaMemcpy(C, C_d, n * sizeof(float), cudaMemcpyDeviceToHost);

  cudaFree(A_d);
  cudaFree(B_d);
  cudaFree(C_d);

}

int main(){
  float arr1[10], arr2[10], arr3[10];
  float *A, *B, *C;
  A = arr1;
  B = arr2;
  C = arr3;

  for (int i = 0; i < 10; i++) {
      arr1[i] = i * 1.1f;
      arr2[i] = i * 2.2f;
  }

  vecAdd(A, B, C, 10);

  return 0;
}




Overwriting vecAdd.cu


In [ ]:
!nvcc vecAdd.cu -o vecAdd

In [ ]:
!./vecAdd